In [27]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import os

In [28]:
path = os.path.normpath(os.getcwd() + os.sep + os.pardir)

In [29]:
def get_flexibility(scenario, demand, location, users2bus):
    flexibilityPath = os.path.join(path, 'Results', 'Generated', f'{location} scenario {scenario}', 'flexibility')
    flex = pd.DataFrame()
    for file in os.listdir(flexibilityPath):
        user = file.split(sep=' ')[0]
        temp = pd.read_csv(os.path.join(flexibilityPath, file), index_col='time')
        flex.insert(0, f'{user}', temp)
        
    # To test the simulation, we are manually increasing the MW amount
    flex = flex * 10
    flex[flex == 0] = 0.001
    flex.index = pd.to_datetime(flex.index)
    
    flex.columns = flex.columns.map(users2bus)
    
    # When the total number of users is greater than the total number of buses, there will be duplicated headers for the flexibility, we sum the total flexibility at each bus here
    flex = flex.groupby(flex.columns, axis=1).sum()
    
    return flex

def get_bus(path, location, state, sub, bus2sub, bus, demand):
    
    sub = sub.loc[sub['name'].str.contains(location) & sub['interconnect'].str.contains(state)]
    user_bus = sub.merge(bus2sub, left_on='sub_id', right_on='sub_id')
    bus2zone = pd.merge(bus, user_bus, on=['bus_id'])
    
    # Maps users to buses
    users2bus = {}
    j = 0
    for i in range(0, len(demand.columns)):
        
        # We only want buses where `Pd` is not 0
        if len(bus2zone['Pd'].loc[bus2zone['Pd'] != 0]) == 1:
            users2bus[demand.columns[i]] = bus2zone['bus_id'].loc[bus2zone['Pd'] != 0].values[0]
        else:
            while bus2zone['Pd'].loc[bus2zone['bus_id'] == user_bus['bus_id'].iloc[j]].values[0] == 0:
                j+=1
            users2bus[demand.columns[i]] = user_bus['bus_id'].iloc[j]
            j += 1
            if j == len(user_bus):
                j = 0
        
    return users2bus, bus2zone

In [30]:
sub = pd.read_csv(os.path.join(path, 'Data', 'Simulation', 'sub.csv'))
bus2sub = pd.read_csv(os.path.join(path, 'Data', 'Simulation', 'bus2sub.csv'))
bus = pd.read_csv(os.path.join(path, 'Data', 'Simulation', 'bus.csv'))
demand = pd.read_csv(os.path.join(path, 'Data', 'nonparticipants.csv'), index_col='time')
demand.index = pd.to_datetime(demand.index)
system_demand = pd.read_csv(os.path.join(path, 'Data', 'Simulation', 'demand.csv'))

In [31]:
# To change the city, location is the name of the substation, the full list is found in the 'sub.csv' file. State is the interconnection name.
# For example, if we want to see the flexibility in Amarillo, since it's in the Panhandle, we would set location to 'PANHANDLE' and leave state as 'Texas'.

location = 'PANHANDLE'
state = 'Texas'
users2bus, bus2zone = get_bus(path, location, state, sub, bus2sub, bus, demand)

flex_up = pd.DataFrame()

for key, item in users2bus.items():
    bus = item
    zone = bus2zone['zone_id'].loc[bus2zone['bus_id'] == bus].values[0]
    
    # Since right now, we don't have demand response data for increase in consumption, we are just setting it to the total demand at the zone level.
    if bus in flex_up.columns:
        continue
    else:
        flex_up. insert(0, bus, system_demand[str(zone)].values)

flex_up.index = system_demand.index

In [32]:
# Reads the lower bound of flexibility
scenario = 3
flexibility_location = 'Amarillo'
flex_dn = get_flexibility(scenario, demand, flexibility_location, users2bus)

flex_up = flex_up[flex_dn.columns]

hydro = pd.read_csv(os.path.join(path, 'Data', 'Simulation', 'hydro.csv'), index_col='UTC')
hydro.index = pd.to_datetime(hydro.index)

flex_up.index = hydro.index
flex_dn.index = hydro.index

# We have found this bus in Austin casues problems with the simulation, so we are removing it from the data set.
if flexibility_location == 'Austin':
    flex_up = flex_up.drop(columns=[3006040])
    flex_dn = flex_dn.drop(columns=[3006040])

In [33]:
os.makedirs(os.path.join(path, 'Results', 'Simulation', flexibility_location))

flex_up.to_csv(os.path.join(path, 'Results', 'Simulation', flexibility_location, 'demand_flexibility_up.csv'))
flex_dn.to_csv(os.path.join(path, 'Results', 'Simulation', flexibility_location, 'demand_flexibility_dn.csv'))

The code above shows how to generate the flexibility files for individual cities for the simulation, now we will simply combine them all together into one file.

To do so, we iterate through the scenario files 

In [34]:
flex_up_full = pd.DataFrame()
flex_dn_full = pd.DataFrame()

for subdir, dirs, files in os.walk(os.path.join(path, 'Results', 'Simulation')):
    for file in files:
        if 'up' in file:
            flex_up_full = pd.concat([flex_up_full, pd.read_csv(os.path.join(subdir, file), index_col=[0])], axis=1)
        if 'dn' in file:
            flex_dn_full = pd.concat([flex_dn_full, pd.read_csv(os.path.join(subdir, file), index_col=[0])], axis=1)

flex_up_full.to_csv(os.path.join(path, 'Results', 'Simulation', 'demand_flexibility_up.csv'))
flex_dn_full.to_csv(os.path.join(path, 'Results', 'Simulation', 'demand_flexibility_dn.csv'))